In [48]:
!pip install pretrainedmodels
!pip install imutils
!pip install opencv-python

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import joblib
import cv2
import os
import time
import random
import pretrainedmodels
import numpy as np

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load torch...!!!
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load torchvision ...!!!
from torchvision import transforms

'''SEED Everything'''
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
SEED=42
seed_everything(SEED=SEED)
'''SEED Everything'''

'SEED Everything'

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # GPU
epochs = 5 # Number of epochs
BS = 16 # Batch size

image_paths = list(paths.list_images('./101_ObjectCategories'))


data = []
labels = []
unique_labels = {}
for img_path in tqdm(image_paths):
    label = img_path.split(os.path.sep)[-2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    
    if label in labels and (unique_labels[label] < 30):
        unique_labels[label] += 1
        data.append(resized)
    elif label not in unique_labels:
        unique_labels[label] = 1
        data.append(resized)
    else:
        continue
        
    labels.append(label)

labels_to_int_dict = {}
labels_to_int = []
next_int = 1

for word in labels:
    if word not in labels_to_int_dict:
        labels_to_int_dict[word] = next_int
        next_int += 1
    labels_to_int.append(labels_to_int_dict[word])

data = np.array(data)
data = data.astype(np.float64) 
labels_to_int = np.array(labels_to_int)
print(data.shape)
print(labels_to_int.shape)

100%|██████████████████████████████████████| 9144/9144 [00:22<00:00, 398.94it/s]


(3060, 128, 128)
(3060,)


In [5]:
lb = LabelEncoder()
labels_to_int = lb.fit_transform(labels_to_int)
print(f"Total Number of Classes: {len(lb.classes_)}")

Total Number of Classes: 102


In [129]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])   

In [8]:
from sklearn.decomposition import PCA

data = data.reshape(data.shape[0], -1)
print(data.shape)

# pca = PCA()
# pca.fit(data)

# # The number of dimensions the training image features should be reduced to, which perserves 95 % of the variances
# variance_ratio = np.cumsum(pca.explained_variance_ratio_)
# num_dimensions = np.argmax(variance_ratio >= 0.95) + 1

# print("Number of dimensions after reduction:", num_dimensions)

num_dimensions = 549
pca = PCA(n_components=num_dimensions)
pca.fit(data)
reduced_data = pca.transform(data)

print(reduced_data.shape)

(3060, 16384)
(3060, 549)


In [11]:
# divide the data into train, validation, and test set
(X, x_val , Y, y_val) = train_test_split(reduced_data, labels_to_int, test_size=0.4,  stratify=labels_to_int,random_state=42)

(x_train, x_test, y_train, y_test) = train_test_split(X, Y, test_size=0.5, random_state=42)
print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nx_val examples: {x_val.shape}")
print(f"y_train examples: {y_train.shape}\ny_test examples: {y_test.shape}\ny_val examples: {y_val.shape}")

x_train examples: (918, 549)
x_test examples: (918, 549)
x_val examples: (1224, 549)
y_train examples: (918,)
y_test examples: (918,)
y_val examples: (1224,)


In [12]:
from scipy import io


x_train = x_train.reshape(x_train.shape[0], -1)
x_train = np.column_stack((x_train, y_train))

x_val = x_val.reshape(x_val.shape[0], -1)
x_val = np.column_stack((x_val, y_val))

x_test = x_test.reshape(x_test.shape[0], -1)
x_test = np.column_stack((x_test, y_test))

print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nx_val examples: {x_val.shape}")

matdata = {
    'labelset': x_train,
    'unlabelset': x_val,
    'testset': x_test
}
io.savemat('./caltech_101.mat', matdata)

# # savetxt('labelset.csv', x_train, delimiter=',')
# # savetxt('unlabelset.csv', x_val, delimiter=',')
# # savetxt('testset.csv', x_test, delimiter=',')

x_train examples: (918, 550)
x_test examples: (918, 550)
x_val examples: (1224, 550)
